In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [2]:
#Please change to your name.
name = "Michael Harris"

#Please use this to control EDA. 
show_eda = False

In [3]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
170,1,0.398,0.711,0.156,0.612,0.054,0.734,0.366,0.388,0.761,...,0.459,0.109,0.093,0.780,0.490,0.980,0.839,0.718,0.395,0.360
207,1,0.988,0.257,0.065,0.737,0.623,0.694,0.983,0.552,0.719,...,0.889,0.508,0.331,0.257,0.984,0.225,0.422,0.512,0.884,0.784
46,1,0.154,0.642,0.770,0.546,0.927,0.092,0.358,0.217,0.853,...,0.953,0.373,0.068,0.551,0.086,0.467,0.550,0.063,0.924,0.080
91,0,0.846,0.089,0.356,0.501,0.415,0.066,0.622,0.368,0.073,...,0.857,0.150,0.771,0.841,0.519,0.761,0.398,0.832,0.179,0.130
155,1,0.208,0.160,0.677,0.947,0.860,0.389,0.685,0.086,0.202,...,0.559,0.949,0.656,0.341,0.550,0.073,0.241,0.959,0.828,0.090


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [4]:
y = np.array(df["target"]).reshape(-1,1)
X = np.array(df.drop(columns={"target"}))
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'DT__max_depth': [None, 10, 20],
    'DT__min_samples_split': [2, 5, 10],
    'DT__min_samples_leaf': [1, 2, 5]
}

grid_search = GridSearchCV(best, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
best_tuned = grid_search.best_estimator_

Best parameters: {'DT__max_depth': 20, 'DT__min_samples_leaf': 5, 'DT__min_samples_split': 2}


In [55]:
from sklearn.ensemble import RandomForestClassifier

scaler = StandardScaler()
estimator = RandomForestClassifier(n_jobs=-1)
pipe = Pipeline(steps=[("scaler", scaler), ("forest", estimator)])

params = {'forest__max_depth':[5,6,7,8,9],
          'forest__min_samples_split':[3,4,5,6,7],
          'forest__min_samples_leaf' :[2,3,4,5,6],
          'forest__n_estimators' :[20,50,80]}
 
best = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1) 
best.fit(X_train, y_train.ravel())
best1 = best.best_estimator_
print(best1.score(X_test, y_test))
print(best1)

0.6507936507936508
Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(max_depth=7, min_samples_leaf=6,
                                        min_samples_split=7, n_estimators=20,
                                        n_jobs=-1))])


In [60]:
clf = DecisionTreeClassifier(max_depth=20,min_samples_split=5, min_samples_leaf=2)
clf = clf.fit(X_train, y_train)
print(clf.get_depth())
print("Training Accuracy:", clf.score(X_train, y_train))
print("Testing Accuracy:", clf.score(X_test, y_test))

8
Training Accuracy: 0.9732620320855615
Testing Accuracy: 0.7142857142857143


In [46]:
#Build pipeline
pipeline_steps = [
    ('scaler', StandardScaler()),
    ('DT', DecisionTreeClassifier(max_depth=20,min_samples_split=5, min_samples_leaf=2))
    ]

best = Pipeline(pipeline_steps)

best.fit(X_train, y_train)

print(best.score(X_train, y_train))
print(best.score(X_test, y_test))
   

0.9732620320855615
0.6825396825396826


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [30]:
print(best.score(X_test, y_test))
print(best)

0.6507936507936508
Pipeline(steps=[('scaler', StandardScaler()),
                ('DT',
                 DecisionTreeClassifier(min_samples_leaf=2,
                                        min_samples_split=5))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [63]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.607426729668189
0.606886075949367
Michael Harris 0.607156402808778


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li>
<li>
</ul>